In [ ]:
%pip install PyCryptodome 

In [ ]:
from Crypto.Util.number import long_to_bytes, inverse
from math import gcd

In [ ]:
N = int("0x7fe8cafec59886e9318830f33747cafd200588406e7c42741859e15994ab62410438991ab5d9fc94f386219e3c27d6ffc73754f791e7b2c565611f8fe5054dd132b8c4f3eadcf1180cd8f2a3cc756b06996f2d5b67c390adcba9d444697b13d12b2badfc3c7d5459df16a047ca25f4d18570cd6fa727aed46394576cfdb56b41", 16)
e = int("0x10001", 16)
c = int("0x5233da71cc1dc1c5f21039f51eb51c80657e1af217d563aa25a8104a4e84a42379040ecdfdd5afa191156ccb40b6f188f4ad96c58922428c4c0bc17fd5384456853e139afde40c3f95988879629297f48d0efa6b335716a4c24bfee36f714d34a4e810a9689e93a0af8502528844ae578100b0188a2790518c695c095c9d677b", 16)

DATA = int("372f0e88f6f7189da7c06ed49e87e0664b988ecbee583586dfd1c6af99bf20345ae7442012c6807b3493d8936f5b48e553f614754deb3da6230fa1e16a8d5953a94c886699fc2bf409556264d5dced76a1780a90fd22f3701fdbcb183ddab4046affdc4dc6379090f79f4cd50673b24d0b08458cdbe509d60a4ad88a7b4e2921", 16)

As the output has the string "RSA broken!?" we can assume from source.py that for DATA:
$$DATA = {DATA}^e \pmod N$$

In [ ]:
print("pow(DATA, e, N) =", pow(DATA, e, N))
print("pow(DATA, e-1, N) =", pow(DATA, e-1, N))

So we can see that:
$${DATA}^{(e-1)} = 1 \pmod N$$
As $e-1 = 65537-1 = 65536 = 2^{16}$ we can try to search if there is a smaller number $k$ such that:
$${DATA}^{k} = 1 \pmod N$$

In [ ]:
for i in range(1,e+2):
    if pow(DATA, i, N) == 1:
        print(i)
        break

So we see that:
$${DATA}^{512} = {DATA}^{2^9} = 1 \pmod N$$
which means:
$${DATA}^{2^9} - 1 = 0 \pmod N \iff {DATA}^{2^9} - 1 = k*N = k*p*q$$
We can use the fact that:
$$
\begin{aligned}
{DATA}^{2^9} - 1 = ({DATA}^{2^8} - 1)({DATA}^{2^8} + 1) &= \\
({DATA}^{2^7} - 1)({DATA}^{2^7} +1 1)({DATA}^{2^8} + 1) &= ...\\
({DATA}^2 - 1)({DATA}^2 + 1)({DATA}^{2^2} + 1)...({DATA}^{2^8} + 1)
\end{aligned}$$
To search if one of those factors is also a factor of $N$.

In [ ]:
for i in range(1, 8):
    if gcd(DATA**(2**i)+1, N) != 1:
        p = gcd(DATA**(2**i)+1, N)
        print(p)

In [ ]:
q = N // p
d = inverse(e, (p-1)*(q-1))
m = pow(c, d, N)
long_to_bytes(m)